In [ ]:
! pip install -U -q langchain, chromadb, langchain-core, langchain-gooogle-vertexai gradio, unstructure, google-cloud-storage, langchain-core

In [ ]:
! source .venv/bin/activate

In [1]:
# import libraries 
from langchain_google_vertexai import VertexAI
from google.cloud import storage
from langchain import LLMChain

In [2]:
# retrievals 
from langchain_community.document_loaders import GCSDirectoryLoader

In [3]:
BUCKET_NAME = "lloyds-genai24lon-2701-bucket"
PROJECT_ID = "lloyds-genai24lon-2701"

In [4]:
llm = VertexAI(model_name = 'text-bison-32k', max_output_tokens = 256, temperature = 0.1, top_p = 0.8, 
               top_k = 40, verbose = True)

In [6]:
loader = GCSDirectoryLoader(project_name = PROJECT_ID, bucket=BUCKET_NAME, prefix="sample/")

In [7]:
documents = loader.load()

In [8]:
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 100

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)

In [11]:
docs = text_splitter.split_documents(documents)
print("splitting...")
print(f"No of Documents = {len(docs)}")

splitting...
No of Documents = 1111


In [14]:
from langchain_google_vertexai import VertexAIEmbeddings

In [ ]:
# Structure ingested documents 

In [15]:
REQUESTS_PER_MINUTE = 500

In [16]:
embeddings = VertexAIEmbeddings(requests_per_minute= REQUESTS_PER_MINUTE)

Model_name will become a required arg for VertexAIEmbeddings starting from Feb-01-2024. Currently the default is set to textembedding-gecko@001


In [17]:
# Vector Store

In [19]:
from langchain_community.vectorstores import Chroma

In [20]:
documents_vector_db = Chroma.from_documents(docs, embeddings)

In [21]:
retriever = documents_vector_db.as_retriever(search_type = 'similarity', search_kwargs = {"k":2})

In [ ]:
# Retrieval QA Chain 

In [22]:
from langchain.chains import RetrievalQA

In [23]:
qa = RetrievalQA.from_chain_type(llm = llm, chain_type = "stuff", retriever=retriever, return_source_documents=True)

### Gradio 

In [34]:
def chatbot(inputtext):
    result = qa({"query": inputtext})
    return result['result'], get_public_url(result['source_documents'][0].metadata['source'], result['source_documents'][0])

In [27]:
def get_public_url(url): 
    components = url.split("/")
    bucket_name = components[2]
    file_name = components[3]
    client = storage.client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(file_name)
    return blob.public_url

In [29]:
import gradio as gr

/home/jupyter/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
examples = ['What is the most common complaint']

In [35]:
iface = gr.Interface(fn=chatbot, 
                     inputs = [gr.Textbox(label="Query")], 
                     examples=examples, title = "FOS chatbot", 
                     outputs = [gr.Textbox(label="Response"), 
                               gr.Textbox(label="URL"), gr.Textbox(label = "Cloud Storage URL")])

In [36]:
iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://668675606f7a03fee3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
